# Preparing VM

In [13]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 24 11:18:43 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:3D:00.0 Off |                    0 |
| N/A   35C    P0    34W / 250W |  23297MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:3E:00.0 Off |                    0 |
| N/A   

# Build service llm

In [ ]:
# ! git clone https://github.com/ggerganov/llama.cpp.git
# % cd llama.cpp
# ! make
# # ! wget https://huggingface.co/openthaigpt/openthaigpt-1.0.0-7b-chat/resolve/main/openthaigpt-Q4_K_M.gguf
# ! ls /opt/data/NLP/llama-index/RAG-for-SuperAI/models/openthaigpt-Q4_K_M.gguf
# ! ./llama-server -m /opt/data/NLP/llama-index/RAG-for-SuperAI/models/openthaigpt-Q4_K_M.gguf -c 1000 --host 0.0.0.0 -ngl 80 --port 8080 -np 2

# Retrieval

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = '../models/bge-m3'
# embedding_model_name = 'kornwtp/simcse-model-phayathaibert'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cpu')

In [2]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [27]:
# model_url = "../models/openthaigpt-Q4_K_M.gguf"
# llm = LlamaCPP(
#     # You can pass in the URL to a GGML model to download it automatically
#     # model_url=model_url,
#     model_path=model_url,
#     # model_path=None,
#     temperature=0.1,
#     max_new_tokens=256,
#     # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
#     context_window=1000,
#     # kwargs to pass to __call__()
#     generate_kwargs={},
#     # kwargs to pass to __init__()
#     # set to at least 1 to use GPU
#     model_kwargs={"n_gpu_layers": 0},
#     # transform inputs into Llama2 format
#     messages_to_prompt=messages_to_prompt,
#     completion_to_prompt=completion_to_prompt,
#     verbose=True,
# )

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/openthaigpt-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32 

In [3]:
from llama_index.core import Settings
# Settings.llm = llm
Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [4]:
from llama_index.core import (

    load_index_from_storage,
    StorageContext,
)

from IPython.display import Markdown, display
from llama_index.vector_stores.faiss import FaissVectorStore


In [5]:
persist_dir="../index"
vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)

In [6]:
query_enging = index.as_query_engine()


In [7]:
question = 'แนะนำที่เที่ยวจังหวัดอยุธยาให้หน่อย'
response_ = query_enging.query(question)

In [8]:
print(response_.response)

Context information is below.
---------------------
==

ศูนย์ท่องเที่ยวอยุธยา (ATC)
ศูนย์ศึกษาประวัติศาสตร์อยุธยา
พิพิธภัณฑสถานแห่งชาติ เจ้าสามพระยา
อุทยานประวัติศาสตร์พระนครศรีอยุธยา
วัดพระศรีสรรเพชญ์
พระราชวังโบราณ อยุธยา
พระที่นั่งวิหารสมเด็จ
พระที่นั่งสรรเพชญปราสาท
พระที่นั่งสุริยาสน์อมรินทร์

เมื่อวันที่ 13 ธันวาคม พ.ศ. 2534 มีพื้นที่ครอบคลุมในบริเวณโบราณสถานเมืองอยุธยา


== ภูมิศาสตร์ ==


=== สภาพภูมิอากาศ ===


== สัญลักษณ์ประจำจังหวัด ==
คำขวัญประจำจังหวัด : ราชธานีเก่า อู่ข้าวอู่น้ำ เลิศล้ำกานท์กวี คนดีศรีอยุธยา เลอคุณค่ามรดกโลก
ตราประจำจังหวัด :
---------------------
Given the context information and not prior knowledge, answer the query.
Query: แนะนำที่เที่ยวจังหวัดอยุธยาให้หน่อย
Answer: 


In [9]:
import json

In [10]:
import requests
def send_api2(prompt,
             url ='http://127.0.0.1:8080/completion',
             temp = 0.7,
             n_predict= 512,
            ):
    payload = json.dumps({
    "prompt": f'''<s>[INST] <<SYS>>
    You are a question answering assistant. Answer the question as truthful and helpful as possible
    คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด
    <</SYS>>

    Answer the question based only on the following context:
    {prompt}

    [/INST]''',
    "n_predict": n_predict,
    'temperature': temp,
    "stop": ["</s>"]
    })

    headers = {
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
#     print(response)
    return response.json()

In [11]:
def generate_answer(question):
    response_ = query_enging.query(question)
    # print(response_.response)
    res_ = send_api2(prompt=response_.response)
    return res_

In [12]:
generate_answer('อยุธยามีกี่อำเภอ')

{'content': 'จังหวัดพระนครศรีอยุธยามีทั้งหมด 16 อำเภอ ได้แก่ อำเภอพระนครศรีอยุธยา อำเภอท่าเรือ อำเภอนครหลวง อำเภอบางปะหัน อำเภอบางไทร อำเภอบ้านแพรก อำเภอบ้านช้าง อำเภอกะเหรี่ยงเฆี่ยน อำเภอบางระจัน อำเภออุทัย อำเภอมโนรมย์ อำเภอนครสวรรค์ อำเภอท่าพระ และอำเภอบางปะอิน',
 'id_slot': 0,
 'stop': True,
 'model': '/opt/data/NLP/llama-index/RAG-for-SuperAI/models/openthaigpt-Q4_K_M.gguf',
 'tokens_predicted': 93,
 'tokens_evaluated': 281,
 'generation_settings': {'n_ctx': 512,
  'n_predict': -1,
  'model': '/opt/data/NLP/llama-index/RAG-for-SuperAI/models/openthaigpt-Q4_K_M.gguf',
  'seed': 4294967295,
  'temperature': 0.699999988079071,
  'dynatemp_range': 0.0,
  'dynatemp_exponent': 1.0,
  'top_k': 40,
  'top_p': 0.949999988079071,
  'min_p': 0.05000000074505806,
  'tfs_z': 1.0,
  'typical_p': 1.0,
  'repeat_last_n': 64,
  'repeat_penalty': 1.0,
  'presence_penalty': 0.0,
  'frequency_penalty': 0.0,
  'penalty_prompt_tokens': [],
  'use_penalty_prompt_tokens': False,
  'mirostat': 0,
  'miros